In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv
import pickle
import copy

from tqdm import tqdm


pd.set_option('display.max_rows', 500)

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline  
import seaborn as sns
sns.set_style("darkgrid")

import umap
from sklearn.decomposition import TruncatedSVD as tsvd

def nearZeroVarDropAuto(df,thresh=0.99):
    vVal=df.var(axis=0).values
    cs=pd.Series(vVal).sort_values(ascending=False).cumsum()
    remove=cs[cs>cs.values[-1]*thresh].index.values
    return df.drop(df.columns[remove],axis=1)

%run SodaKick_download_functions.ipynb


import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import SGD, Adagrad, Adam, Adagrad
import torch.nn as nn
import torch.nn.functional as F

In [3]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [4]:
class EarlyStopping:

    """ Stops the training if loss doesn't improve after a given number of epochs. """

    def __init__(self, patience=3, epsilon=1e-5, keepBest=True, silent=True):

        """
        Args:
            patience (int): Number of epochs without change before stopping the learning (default 3).
            epsilon (float): Minimum change in loss to be considered for early stopping (default 1e-5).
            keepBest (bool): Keep track of the best model (memory consuming).
        """

        self.patience = patience
        self.epsilon = epsilon
        self.counter = 0
        
        self.bestScore = np.inf
     
        self.keepBest = keepBest 
        self.bestModel = None

        self.earlyStop = False
        self.silent = silent

    def __call__(self, loss, model):


        """ Evaluate the loss change between epochs and activates early stop if below epsilon.

        Args:
            loss (float): current loss.
            model (torch model): the current model.
        """

        if loss > self.bestScore - self.epsilon:

            self.counter += 1
            if not self.silent:
                print('EarlyStopping counter: {:d}/{:d}'.format(self.counter,self.patience))

            if self.counter >= self.patience:
                self.earlyStop = True

        else:   

            self.counter = 0
            self.bestScore = loss

            if self.keepBest:
                self.bestModel = copy.deepcopy(model)


In [16]:
class matchesDataset(Dataset):

    """ Extend pytorch Dataset class to include cleaning and training set creation, """
    
    def __init__(self, matches, results):

        self.matches = torch.tensor(matches, dtype=torch.float32)
        self.results = torch.tensor(results, dtype=torch.float32)

    def __len__(self):

        """ Returns the len of the training sample. """
        
        return len(self.matches)
        

    def __getitem__(self, index): 

        """ Returns a word, a context word and a list of negative words for training for a given index. 

        Args:
            index (int): index for the word selection.

        Returns:
            (string, string, list of strings): selected word, context word and a randomly drawn list 
                                               of negative words.

        """
        return self.matches[index], self.results[index]

In [6]:
#https://towardsdatascience.com/quirky-keras-custom-and-asymmetric-loss-functions-for-keras-in-r-a8b5271171fe
def WSE(output, target, a=1.5, b=.5):
    loss = torch.mean(a*torch.minimum(torch.zeros(output.shape[1]),output - target)**2+\
                      b*torch.maximum(torch.zeros(output.shape[1]),output - target)**2)      
    return loss

def WSEl1(output, target, a=1.5, b=.5):
    loss = torch.mean(a*torch.abs(torch.minimum(torch.zeros(output.shape[1]),output - target))+\
                      b*torch.abs(torch.maximum(torch.zeros(output.shape[1]),output - target)))      
    return loss

def WSE2(output, target, a=1.5, b=.5):
    loss = np.mean(a*np.minimum(np.zeros(output.shape[0]),output - target)**2+\
                      b*np.maximum(np.zeros(output.shape[0]),output - target)**2)      
    return loss

def WSEl12(output, target, a=1.5, b=.5):
    loss = np.mean(a*np.abs(np.minimum(np.zeros(output.shape[0]),output - target))+\
                      b*np.abs(np.maximum(np.zeros(output.shape[0]),output - target)))      
    return loss

In [7]:
from sklearn import preprocessing

    
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_inp_2a.pkl', 'rb') as pk:
    inp=pickle.load(pk)
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_out_2a.pkl', 'rb') as pk:
    out=pickle.load(pk)     
    
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_inp_2b.pkl', 'rb') as pk:
    inpb=pickle.load(pk)
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_out_2b.pkl', 'rb') as pk:
    outb=pickle.load(pk)     

inp=np.concatenate([inp,inpb])
out=np.concatenate([out,outb])

def normalize_mins(vec):
    for i in range(vec.shape[0]):
        vec[i][::8]=vec[i][::8]/90

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def NormalizeMatrix(data):   
    for i in range(data.shape[1]):
        data[:,i] = NormalizeData(data[:,i])
        
NormalizeMatrix(inp)
np.nan_to_num(inp, copy=False)

normalize_mins(out)

#scaler = preprocessing.StandardScaler().fit(inp)
#inp = scaler.transform(inp)

//miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


In [14]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
         inp, out, test_size=0.2, random_state=32)

In [13]:
class Net(nn.Module):
    
    def __init__(self, num_layers, num_nodes, scaling_factor, num_nodes_out, final_activation):
        super(Net1, self).__init__()
        
        self.fc = []
        self.lr = []
        self.fact = final_activation
        self.nl = num_layers
        power=0
        
        for i in range(self.nl):
            self.fc.append(nn.Linear(num_nodes*scaling_factor**power, num_nodes*scaling_factor**(power+1)))
            self.lr.append(nn.LeakyReLU())
            power+=1
        
        self.oupt = nn.Linear(num_nodes*scaling_factor**power, num_nodes_out)
    
    def forward(self, x):
        z = x
        for i in range(self.nl):
            z = self.lr[i](self.fc[i](z))
        z = self.oupt(self.fact(z))
        return z
    
    def clp(self):
        with torch.no_grad():
            for i in range(self.nl):
                self.fc[i].weight.copy_ (self.fc[i].weight.data.clamp(min=0)) 
            self.oupt.weight.copy_ (self.oupt.weight.data.clamp(min=0))

In [55]:
def train(config, model=Net, silent=True, checkpoint_dir=None):
    
    phases = ['train','val']
    
    #x_train, x_test, y_train, y_test = data[0], data[1], data[2], data[3]
    
    training_set = matchesDataset(x_train, y_train)
    trainBatch = torch.utils.data.DataLoader(training_set, batch_size=config['batch_size'], shuffle=config['shuffle'], num_workers=config['num_workers'])

    validation_set = matchesDataset(x_test, y_test)
    valBatch = torch.utils.data.DataLoader(validation_set, batch_size=config['batch_size'], shuffle=config['shuffle'], num_workers=config['num_workers'])

    earlStop = EarlyStopping(patience=config['patience'], keepBest=False)
    net = model(config)
    
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    if config['optim']=='adam':
        optimizer = Adam(net.parameters(), lr=config['lr'])
    elif config['optim']=='adagrad':
        optimizer = Adagrad(net.parameters(), lr=config['lr'])
    else:
        print('optim error')
        return
    
    
    losses=[[],[]]
    mses=[]
    diffs=[]
    exit=False
    
    #for epoch in tqdm(range(epochs), desc='Epoch'):
    for epoch in range(epochs):

        if exit:
            break
            
        for phase in phases:
            if phase == 'train':
                net.train(True) 

                """ Run the training of the model. """    

                losses_batch=[]
                for batchNum, batch in enumerate(trainBatch):

                    x = batch[0]
                    y = batch[1]

                    """ Move batches to GPU if available. """

                    if torch.cuda.is_available():
                        x = x.cuda()
                        y = y.cuda()

                    """ Core of training. """
                    
                    loss = config['loss_f'](net(x), y)

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    if config['clip']:
                        net.clp()

                    losses_batch.append(loss)

                """ Early stop check. """

                earlStop(loss, net)
                finalepoch = epoch
                
                if earlStop.earlyStop:

                    if not silent:
                        print('Limit loss improvement reached, stopping the training.')
                        
                    exit=True 
                
                losses[0].append(torch.mean(torch.stack(losses_batch)).detach().cpu().numpy())
            else:
                net.train(False)
                net.eval()
                
                val_loss=0
                val_mse=0
      
                losses_batch=[]
                for batchNum, batch in enumerate(valBatch):

                    x = batch[0]
                    y = batch[1]

                    """ Move batches to GPU if available. """

                    if torch.cuda.is_available():
                        x = x.cuda()
                        y = y.cuda()

                    """ Core of training. """

                    optimizer.zero_grad()
                    output=net(x)
                    target=y
                    loss = config['loss_f'](output, target)

                    #losses_batch.append(loss)
                    val_loss+=loss
                    val_mse+=nn.MSELoss()(output, target)
                    
                losses[1].append(torch.mean(torch.stack(losses_batch)).detach().cpu().numpy())
                
                with tune.checkpoint_dir(epoch) as checkpoint_dir:
                    path = os.path.join(checkpoint_dir, "checkpoint")
                    torch.save((net.state_dict(), optimizer.state_dict()), path)

                tune.report(loss=(val_loss/batchNum), mse=(val_mse/batchNum))
                #tune.report(loss=torch.mean(torch.stack(losses_batch)))

In [57]:
tune.checkpoint_dir(0)

In [18]:
def revert_output(output,lineup):

    reframe=pd.DataFrame(output.reshape(50,8), index=lineup.index,
                 columns=['minutes','goals','assists','cards_yellow','cards_red','own_goals','goals_against','saves'])
    reframe.drop([x for x in reframe.index if x.startswith('dummy')], axis=0, inplace=True)
    byteamframe=pd.concat([reframe.loc[[x for x in reframe.index if x in lineup[lineup['team']==0].index]].sum(axis=0),
                        reframe.loc[[x for x in reframe.index if x in lineup[lineup['team']==1].index]].sum(axis=0)], axis=1).T
    
    reframe['minutes']=reframe['minutes']*90
    
    return reframe, byteamframe[byteamframe.columns[1:]]
    

In [11]:
print('Baseline WSE: {:.3f}'.format(WSE2(np.array([0]*out[0].shape[0]),out[0])))
print('Baseline WSE l1: {:.3f}'.format(WSEl12(np.array([0]*out[0].shape[0]),out[0])))
print('Baseline MSE: {:.3f}'.format(WSE2(np.array([0]*out[0].shape[0]),out[0], a=1, b=1)))
print('Baseline MSE l1: {:.3f}'.format(WSEl12(np.array([0]*out[0].shape[0]),out[0], a=1, b=1)))

print((out[1]-out[10]).sum())
print((out[50]-out[60]).sum())
print((out[100]-out[110]).sum())

Baseline WSE: 0.230
Baseline WSE l1: 0.139
Baseline MSE: 0.153
Baseline MSE l1: 0.092
-18.0
16.566666666666666
2.0000000000000004


In [21]:
tune.loguniform(1e-4, 1e-1)

In [43]:
int(5000*1.0001)

5000

In [53]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    

    config = {
        "num_layers": tune.choice([1, 2, 3]),
        "num_nodes": inp.shape[1],
        "scaling_factor": tune.sample_from(lambda _: np.random.uniform(low=0.5, high=1.0001)),
        "num_nodes_out": out.shape[1], 
        "final_activation" : torch.tanh,
        "clip": tune.choice([True, False]),
        "batch_size": tune.choice([8, 16, 32]),
        "loss_f": tune.choice([WSE, nn.MSELoss(), nn.L1Loss()]),
        "optim": tune.choice(['adam', 'adagrad']),
        "lr": 1e-3,
        "shuffle": True,
        "num_workers": 4,
        "patience": 10,
    }
    
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    
    reporter = CLIReporter(
        metric_columns=["loss", "mse", "training_iteration"])
    
    result = tune.run(
        train,
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation mse: {}".format(
        best_trial.last_result["mse"]))

    best_trained_model = Net(best_trial.config)
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    #test_acc = test_accuracy(best_trained_model, device)
    #print("Best trial test set accuracy: {}".format(test_acc))

In [54]:
main(num_samples=10, max_num_epochs=10, gpus_per_trial=0)

2021-05-26 15:26:03,510	WARNING worker.py:1115 -- Warning: The actor ImplicitFunc has size 84675864 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2021-05-26 15:26:03,606	WARNING util.py:162 -- The `start_trial` operation took 0.710 s, which may be a performance bottleneck.


== Status ==
Memory usage on this node: 9.0/16.0 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/8 CPUs, 0/0 GPUs, 0.0/4.04 GiB heap, 0.0/2.02 GiB objects
Result logdir: /Users/federico comitani/ray_results/train_2021-05-26_15-25-58
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------+----------+-------+--------------+--------+-------------------------------+--------------+---------+------------------+
| Trial name        | status   | loc   |   batch_size | clip   | loss_f                        |   num_layers | optim   |   scaling_factor |
|-------------------+----------+-------+--------------+--------+-------------------------------+--------------+---------+------------------|
| train_32f52_00000 | RUNNING  |       |           16 | False  | MSELoss()                     |            3 | adagrad |         0.732012 |
| train_32f52_00001 | PENDING  |       |           16 | 

2021-05-26 15:26:04,242	WARNING util.py:162 -- The `start_trial` operation took 0.606 s, which may be a performance bottleneck.
2021-05-26 15:26:04,890	WARNING util.py:162 -- The `start_trial` operation took 0.647 s, which may be a performance bottleneck.
2021-05-26 15:26:05,584	WARNING util.py:162 -- The `start_trial` operation took 0.693 s, which may be a performance bottleneck.
(pid=74303) 2021-05-26 15:26:06,120	ERROR function_runner.py:254 -- Runner Thread raised error.
(pid=74303) Traceback (most recent call last):
(pid=74303)   File "/Users/federico comitani/.local/lib/python3.7/site-packages/ray/tune/function_runner.py", line 248, in run
(pid=74303)     self._entrypoint()
(pid=74303)   File "/Users/federico comitani/.local/lib/python3.7/site-packages/ray/tune/function_runner.py", line 316, in entrypoint
(pid=74303)     self._status_reporter.get_checkpoint())
(pid=74303)   File "/Users/federico comitani/.local/lib/python3.7/site-packages/ray/tune/function_runner.py", line 580, i

Result for train_32f52_00000:
  {}
  


(pid=74304) 2021-05-26 15:26:06,571	ERROR function_runner.py:254 -- Runner Thread raised error.
(pid=74304) Traceback (most recent call last):
(pid=74304)   File "/Users/federico comitani/.local/lib/python3.7/site-packages/ray/tune/function_runner.py", line 248, in run
(pid=74304)     self._entrypoint()
(pid=74304)   File "/Users/federico comitani/.local/lib/python3.7/site-packages/ray/tune/function_runner.py", line 316, in entrypoint
(pid=74304)     self._status_reporter.get_checkpoint())
(pid=74304)   File "/Users/federico comitani/.local/lib/python3.7/site-packages/ray/tune/function_runner.py", line 580, in _trainable_func
(pid=74304)     output = fn()
(pid=74304)   File "<ipython-input-50-db55d1fe5971>", line 14, in train
(pid=74304) NameError: name 'model' is not defined
(pid=74304) Exception in thread Thread-2:
(pid=74304) Traceback (most recent call last):
(pid=74304)   File "//miniconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
(pid=74304)     self.run()
(pid=

...

Result for train_32f52_00003:
  {}
  


2021-05-26 15:26:10,147	WARNING util.py:162 -- The `start_trial` operation took 0.643 s, which may be a performance bottleneck.
2021-05-26 15:26:10,329	WARNING tune.py:507 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2021-05-26 15:26:10,581	ERROR trial_runner.py:732 -- Trial train_32f52_00004: Error processing event.
Traceback (most recent call last):
  File "/Users/federico comitani/.local/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 702, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "/Users/federico comitani/.local/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 686, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "/Users/federico comitani/.local/lib/python3.7/site-packages/ray/_private/client_mode_hook.py", line 47, in wrapper
    

Result for train_32f52_00004:
  {}
  
== Status ==
Memory usage on this node: 9.5/16.0 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 6.0/8 CPUs, 0/0 GPUs, 0.0/4.04 GiB heap, 0.0/2.02 GiB objects
Result logdir: /Users/federico comitani/ray_results/train_2021-05-26_15-25-58
Number of trials: 10/10 (5 ERROR, 2 PENDING, 3 RUNNING)
+-------------------+----------+-------+--------------+--------+-------------------------------+--------------+---------+------------------+
| Trial name        | status   | loc   |   batch_size | clip   | loss_f                        |   num_layers | optim   |   scaling_factor |
|-------------------+----------+-------+--------------+--------+-------------------------------+--------------+---------+------------------|
| train_32f52_00005 | RUNNING  |       |           32 | True   | L1Loss()                      |            3 | adagrad |         0.890031 |
| train_3

2021-05-26 15:26:10,844	ERROR tune.py:545 -- Trials did not complete: [train_32f52_00000, train_32f52_00001, train_32f52_00002, train_32f52_00003, train_32f52_00004, train_32f52_00005, train_32f52_00006, train_32f52_00007, train_32f52_00008, train_32f52_00009]
2021-05-26 15:26:10,844	INFO tune.py:549 -- Total run time: 12.64 seconds (11.96 seconds for the tuning loop).
2021-05-26 15:26:10,845	WARNING tune.py:554 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
2021-05-26 15:26:10,853	WARNING experiment_analysis.py:580 -- Could not find best trial. Did you pass the correct `metric` parameter?


AttributeError: 'NoneType' object has no attribute 'config'

In [244]:
pred=nt(torch.Tensor(inptmp)).detach().cpu().numpy()[3]
cats=['minutes','goals','assists','cards_yellow','cards_red','own_goals']+['goals_against','saves']

reframe, byteamframe = revert_output(pred)
print(byteamframe)
reframe, byteamframe = revert_output(out[3])
print(byteamframe)

      goals   assists  cards_yellow  cards_red  own_goals  goals_against  \
0  1.571054  1.131036      2.305363   0.181507   0.124638       1.737184   
1  1.362419  1.026369      2.779735   0.322705   0.215665       1.503933   

      saves  
0  2.677974  
1  3.405122  
   goals  assists  cards_yellow  cards_red  own_goals  goals_against  saves
0    4.0      4.0           4.0        0.0        0.0            1.0    2.0
1    1.0      1.0           2.0        0.0        0.0            4.0    2.0
